In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.pandas.set_option('display.max_columns',None)
pd.pandas.set_option('display.max_rows',None)

import datetime as datetime


import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go


import matplotlib.pyplot as plt
import calendar
from datetime import datetime


In [2]:
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
def get_choice(options):
    # Display choices
    print("Please select one of the following options:")
    for idx, option in enumerate(options, 1):
        print(f"{idx}. {option}")

    # Get user input
    while True:
        try:
            choice = int(input("Enter the number of your choice: "))
            if 1 <= choice <= len(options):
                return options[choice - 1]
            else:
                print("Invalid choice. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# Example usage


In [4]:
df=pd.read_csv('model_dataset.csv')

In [5]:
data=df.copy()
data['Date'] = pd.to_datetime(data['Date'])
data['year']=data['Date'].dt.year

data['Card Number']=data['Card Number'].astype(np.int64)
data['year']=data['year'].astype('str')
data['MCC']=data['MCC'].astype('str')
data['Is Fraud?']=data['Is Fraud?'].map({0: 'Non_Fraudulent', 1: 'Fraudulent'})
data['Has Chip']=data['Has Chip'].map({0: 'NO', 1: 'Yes'})

In [6]:
dfas=data[data['Is Fraud?']=='Fraudulent']

In [7]:
def visualization(das_df,card_number):    
    try:              
        x=int(card_number)        
        data=das_df[das_df['Card Number']==x] 
        #Data disribution
        
        labels = data['Is Fraud?'].value_counts().index
        values = data['Is Fraud?'].value_counts().values
        fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
        fig.update_layout(
            title="Credit Card transaction Data Distribution".upper(),
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#000000"
            )
        )
        
        fig.show()


        labels = data['Has Chip'].value_counts().index
        values = data['Has Chip'].value_counts().values
        fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
        fig.update_layout(
            title="Analysis of chip in card".upper(),
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#000000"
            )
        )
        
        fig.show()
        
          
        df1=pd.DataFrame()
        df1['year']=data['year'].unique().tolist()
        
        df1.sort_values(by='year', inplace = True)
        
        df1['year']=df1['year'].astype(str)
        df1=df1.reset_index()
        del df1['index']
        
        da=data.groupby(by=['year','Is Fraud?']).size().reset_index(name='credit_card_Transactions')
        da['year']=da['year'].astype(str)
        
        Fraud_trans=[]
        Non_Fraud_trans=[]
        for i in range(df1.shape[0]):
            da1=da[da['year']==df1['year'][i]]
            if da1.shape[0]>0:
                if da1[da1['Is Fraud?']=='Non_Fraudulent'].shape[0]>0:                    
                    Non_Fraud_trans.append(da1[da1['Is Fraud?']=='Non_Fraudulent']['credit_card_Transactions'].tolist()[0])
                else:
                    Non_Fraud_trans.append(0)
                if da1[da1['Is Fraud?']=='Fraudulent'].shape[0]>0:
                    Fraud_trans.append(da1[da1['Is Fraud?']=='Fraudulent']['credit_card_Transactions'].tolist()[0])
                else:
                    Fraud_trans.append(0)
        df1['Non_Fraudulent_trans']=Non_Fraud_trans 
        df1['Fraudulent_trans']=Fraud_trans
        a=df1['year'].values
        b=Non_Fraud_trans
        c=Fraud_trans
        
        fig = go.Figure(
        data=go.Scatter(x=a,y=b,name="Non_Fraudulent Transactions",marker=dict(color="blue"),mode="lines+text",text=b,textposition="top right"))
    
        fig.adf_trace(go.Bar(x=a,y=c,yaxis="y2",name="Fraudulent Transactions",marker=dict(color="red"),text=c,textposition="inside",))
    
        fig.update_layout(legend=dict(orientation="h"),yaxis=dict(title=dict(text="Non_Fraudulent Transactions"),side="left",),
            yaxis2=dict(title=dict(text="Fraudulent Transactions"),side="right",overlaying="y",tickmode="sync",),
            width=1100,
            height=500,
            title='credit card data distribution'.upper()
        )
        
        fig.show()   


        df=data.groupby('Use Chip')['Use Chip'].agg(['count']).sort_values(by='count',ascending=False)[:10]
        df=df.reset_index()
        fig=px.bar(data_frame=df,x='Use Chip',y='count',color='Use Chip',barmode='group',text_auto=True,title='Analysis of Types of transactions'.upper())
        fig.show()

        df=data.groupby(by=['Use Chip','Is Fraud?']).size().reset_index(name='No. of Transactions')
        fig1=px.bar(data_frame=df,x='Use Chip',y='No. of Transactions',color='Is Fraud?',barmode='group',text_auto=True,title='Trans.types/is_fraud wise Data detection'.upper())
        fig1.show()

        
        df=data.groupby('MCC')['MCC'].agg(['count']).sort_values(by='count',ascending=False)[:10]
        df=df.reset_index()
        fig=px.bar(data_frame=df,x='MCC',y='count',color='MCC',barmode='group',text_auto=True,title='Category code card Wise Fraud Detection'.upper())
        fig.show()

        df1=data.copy()
        df1.loc[df1['Amount'].between(-1000,0), 'amount_range'] = '-1k-0k'
        df1.loc[df1['Amount'].between(0,200), 'amount_range'] = '0-0.2k'
        df1.loc[df1['Amount'].between(200,400), 'amount_range'] = '0.2k-0.4k'
        df1.loc[df1['Amount'].between(400,600), 'amount_range'] = '0.4k-0.6k'
        df1.loc[df1['Amount'].between(600,800), 'amount_range'] = '0.6k-0.8k'
        df1.loc[df1['Amount'].between(800,1000), 'amount_range'] = '0.8k-1k'
        df1.loc[df1['Amount'].between(1000,1200), 'amount_range'] = '1k-1.2k'
        df1.loc[df1['Amount'].between(1200,1400), 'amount_range'] = '1.2k-1.4k'
        df1.loc[df1['Amount'].between(1400,1600), 'amount_range'] = '1.4k-1.6k'
        df1.loc[df1['Amount'].between(1600,1800), 'amount_range'] = '1.6k-1.8k'
        df1.loc[df1['Amount'].between(1800,2000), 'amount_range'] = '1.8k-2k'
        df1.loc[df1['Amount']>2000, 'amount_range'] = '>2k'
        df_d=df1.copy()
        df1=df1.groupby(by=['amount_range','Is Fraud?']).size().reset_index(name='No. of Transactions')
        fig1=px.bar(data_frame=df1,x='amount_range',y='No. of Transactions',color='Is Fraud?',barmode='group',text_auto=True,title='amount_range/is_fraud wise Data detection'.upper())
        fig1.show()

        df=df_d.groupby(by=['Use Chip','amount_range']).size().reset_index(name='No. of Incidents')
        fig20=px.bar(data_frame=df,x='amount_range',y='No. of Incidents',color='Use Chip',barmode='group',title='types of transactions/is_fraud wise Fraud detection'.upper(),text_auto=True)
        fig20.show()

        fig = px.sunburst(
            df_d,
            path=["Card Number", 'Card Brand',"Has Chip","Is Fraud?","Use Chip","MCC",'amount_range'],
            title="Overall Data Distribution",
            color="Is Fraud?",
            height=800,
        )
            # display the figure
        fig.show()
 
    except Exception as e:
        print('please enter valid credit card number')

In [8]:
with open(r'training_dataMerchant State.pkl', 'rb') as file:
    merchant_state_label_encoder = pickle.load(file)

In [9]:
with open(r'training_dataMerchant City.pkl', 'rb') as file:
    Merchant_City_label_encoder = pickle.load(file)

In [10]:
with open(r'training_dataCard Type.pkl', 'rb') as file:
    Card_Type_label_encoder = pickle.load(file)

In [11]:
scaler1 = pickle.load(open(r'scaled_data.sav', 'rb'))

In [12]:
rfc=pickle.load(open(r'casa_prediction.sav','rb'))

In [13]:
please enter the card number: 4879494103069057
please enter the Credit Limit: 12400
please enter the mmc number: 5411
please enter the FICO Score: 787
please enter the transaction amount: 130.23
please enter the Yearly Income: 59696
please enter the Current Age: 53

SyntaxError: invalid syntax (1028190804.py, line 1)

In [28]:
def information():
    try:
        Card_Number=np.array([int(input('please enter the card number:'))])
        
        data_shape=data[data['Card Number']==Card_Number[0]].shape[0]
        if data_shape>0:
            FICO_Score=np.array([int(input('please enter the FICO Score:'))])

            Yearly_Income=np.array([int(input('please enter the Yearly Income:'))])

            Current_Age=np.array([int(input('please enter the Current Age:'))])
            
            Credit_Limit=np.array([int(input('please enter the Credit Limit:'))])
            
            mcc=np.array([int(input('please enter the MCC number:'))])           
            
            amount=np.array([float(input('please enter the transaction amount:'))])           
            
            use_chip_opt=['Swipe Transaction','Chip Transaction','Online Transaction']
            use_chip=get_choice(use_chip_opt)
            use_chip_map={'Swipe Transaction':[1], 'Chip Transaction':[2], 'Online Transaction':[3]}
            use_chip=use_chip_map.get(use_chip, "Invalid input")
            
            Card_rand_opt=['Visa','Amex','Mastercard','Discover']
            Card_Brand=get_choice(Card_rand_opt)
            Card_Brand_map={'Visa':[1], 'Amex':[2], 'Mastercard':[3], 'Discover':[4]}
            Card_Brand=Card_Brand_map.get(Card_Brand, "Invalid input")

            Card_Type_opt=['Debit','Debit(Prepaid)','Credit']
            Card_Type=get_choice(Card_Type_opt)
            Card_Type_map={'Debit':[1], 'Debit(Prepaid)':[2], 'Credit':[3]}
            Card_Type=Card_Type_map.get(Card_Type, "Invalid input")

            merchant_state=merchant_state_label_encoder.transform([str(input('please enter merchant state:-')).upper().strip()])
            
            Merchant_City=Merchant_City_label_encoder.transform([str(input('please enter merchant city:-')).strip()])
            
            
            options=['YES','NO']
            Has_Chip=get_choice(options)
            opt_map={'YES':[1], 'NO':[0]}
            Has_Chip=opt_map.get(Has_Chip, "Invalid input")
            
            Card_Index=np.array([float(input('please enter the Card Index:'))])
            
            user_Latitude=np.array([float(input('please enter the user Latitude:'))])
            
            user_Longitude=np.array([float(input('please enter the user Longitude:'))])
    
            print('\033[92m\033[1m THE VISUALISATIONS OF PAST CARD TRANSACTIONS:- ')
            visualization(data,Card_Number[0])
    
            input_d=np.concatenate((amount,use_chip,Merchant_City,merchant_state,mcc,Card_Brand,Card_Type,Card_Number,Has_Chip,Credit_Limit,Current_Age,user_Latitude,user_Longitude,Yearly_Income,FICO_Score))
            input_d=np.concatenate((amount,use_chip,Merchant_City,merchant_state,mcc,Current_Age,user_Latitude,user_Longitude,Yearly_Income,FICO_Score,Card_Index,Card_Brand,Card_Type,Card_Number,Has_Chip,Credit_Limit))
            input1=input_d.astype(np.float64) 
            input_data=input1.reshape(1, -1)
    
        else:
            print('Oh! Sorry. Please enter Correct card number')
    except Exception as e:
        print('\033[91m\033[1mSorry. Please enter valid credit card number or Proper details')
        input_data=''
    return input_data

In [18]:
dfas.head(1)

,Unnamed: 0,Date,Amount,Use Chip,Merchant City,Merchant State,MCC,Is Fraud?,Current Age,Gender,Latitude,Longitude,Yearly Income - Person,Total Debt,FICO Score,CARD INDEX,Card Brand,Card Type,Card Number,Has Chip,Cards Issued,Credit Limit,year
0,0,2015-11-15,287.13,Online Transaction,ONLINE,ONLINE,3001,Fraudulent,53,Female,34.15,-117.76,59696,127613,787,0,Visa,Debit,4344676511950444,Yes,2,24295,2015


In [29]:
xx=information()

Please select one of the following options:
1. Swipe Transaction
2. Chip Transaction
3. Online Transaction
Please select one of the following options:
1. Visa
2. Amex
3. Mastercard
4. Discover
Please select one of the following options:
1. Debit
2. Debit(Prepaid)
3. Credit
Please select one of the following options:
1. YES
2. NO
 THE VISUALISATIONS OF PAST CARD TRANSACTIONS:- 
please enter valid credit card number


In [21]:
xx=information()

Please select one of the following options:
1. Swipe Transaction
2. Chip Transaction
3. Online Transaction
Please select one of the following options:
1. Visa
2. Amex
3. Mastercard
4. Discover
Please select one of the following options:
1. Debit
2. Debit(Prepaid)
3. Credit
Please select one of the following options:
1. YES
2. NO
 THE VISUALISATIONS OF PAST CARD TRANSACTIONS:- 
please enter valid credit card number


In [ ]:
red = '\033[91m'
green = '\033[92m'
blue = '\033[94m'
bold = '\033[1m'
italics = '\033[3m'
underline = '\033[4m'
end = '\033[0m'

In [30]:
xx

array([[ 2.87130000e+02,  3.00000000e+00,  2.60600000e+03,
         6.80000000e+01,  3.00100000e+03,  5.30000000e+01,
         3.41500000e+01, -1.17760000e+02,  5.96960000e+04,
         7.87000000e+02,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  4.34467651e+15,  1.00000000e+00,
         2.42950000e+04]])

In [31]:
X_test_initial= scaler1.transform(xx)

c:\Users\bharath.venkatesan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



In [32]:
X_test_initial

array([[ 1.32815735,  1.21569687,  0.33557832,  0.56803792, -2.51759799,
        -0.22376   , -0.58133998, -1.54364486,  0.65739627,  1.08652565,
        -1.06602443, -1.25099825,  0.39424676, -0.34576871,  0.32235086,
         0.90604373]])

In [33]:
x=rfc.predict(X_test_initial)

In [34]:
if x[0]==0:
    print('\033[1m\033[92mThe transaction is not a fraudlent transaction')
else:
    print('\033[1m\033[91mSorry! The transaction may be fraudlent transaction')

Sorry! The transaction may be fraudlent transaction
